<a href="https://colab.research.google.com/github/Dharshan4038/Invoice_Purchase_Order_Match/blob/main/INVOICE_PO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Invoice_PO_Match/validation_formatted 1.csv')
df.head()

,invoice_id,invoice_desc,PO_line_desc,true_mapping
0,INV-49266647,"['25103 ICE CREAM MANGO SORBETTO TRAY', 'ICE C...","['ICE CREAM MANGO SORBETTO TRAY', 'ICE CREAM G...",[{'25103 ICE CREAM MANGO SORBETTO TRAY': 'ICE ...
1,INV-49266660,"['ICE CREAM GELATO MACKINAC FDGE 3/5LT', 'ICE ...","['ICE CREAM GELATO MACKINAC FDGE', 'ICE CREAM ...",[{'ICE CREAM GELATO MACKINAC FDGE 3/5LT': 'ICE...
2,INV-49266724,['Scotch-Brite Quick Clean Griddle Liquid 700-...,"['CLEANER GRL QUICKCLEAN PCKT', 'PAD SCOUR COM...",[{'Scotch-Brite Quick Clean Griddle Liquid 700...
3,INV-49267382,"['Jersey Mikes Mini Wheat Sub', 'Jersey Mikes'...","['ROLL MINI WHEAT SUB', 'ROLL MINI WHT SUB', '...",[{'Jersey Mikes Mini Wheat Sub': 'ROLL MINI WH...
4,INV-49267510,"['CASES BEEF GRND CHUCK PATTY 4oz', '14 CASES ...","['BEEF GRND CHUCK PATTY 4OZ', 'BEEF GRND THICK...",[{'CASES BEEF GRND CHUCK PATTY 4oz': 'BEEF GRN...


In [2]:
invoice = df["invoice_desc"]
purchase_order = df["PO_line_desc"]
true_mapping = df["true_mapping"]

In [3]:
import ast
for i in range(len(invoice)):
    invoice[i] = ast.literal_eval(invoice[i])
    purchase_order[i] = ast.literal_eval(purchase_order[i])
    true_mapping[i] = eval(true_mapping[i])

In [4]:
# Initialize an empty list to store all rows for the DataFrame
final_data = []

# Loop through each set of invoices, purchase_orders, and true_mapping for each row
for invoice_list, po_list, mapping in zip(invoice, purchase_order, true_mapping):

    # Create a dictionary for true mappings of each row
    mapping_dict = {list(d.keys())[0]: list(d.values())[0] for d in mapping}

    # Generate all combinations of invoice and purchase orders for each row
    for invoice in invoice_list:
        for po in po_list:
            # Check if the current pair is correct according to true_mapping
            label = 1 if mapping_dict.get(invoice) == po else 0
            final_data.append([invoice, po, label])


In [5]:
df = pd.DataFrame(final_data, columns=['Invoice', 'Purchase Order', 'Label'])
df.head(20)

,Invoice,Purchase Order,Label
0,25103 ICE CREAM MANGO SORBETTO TRAY,ICE CREAM MANGO SORBETTO TRAY,1
1,25103 ICE CREAM MANGO SORBETTO TRAY,ICE CREAM GELATO MACKINAC FDGE,0
2,25103 ICE CREAM MANGO SORBETTO TRAY,ICE CREAM COOKIE%CRM DUTCH FUD,0
3,25103 ICE CREAM MANGO SORBETTO TRAY,ICE CREAM GELATO BUTR PECAN,0
4,25103 ICE CREAM MANGO SORBETTO TRAY,ICE CREAM SORBET LEMONCELLO,0
5,25103 ICE CREAM MANGO SORBETTO TRAY,ICE CREAM GELATO CHOC ND VEGAN,0
6,ICE CREAM GELATO MACKINAC FDGE 3/5LT,ICE CREAM MANGO SORBETTO TRAY,0
7,ICE CREAM GELATO MACKINAC FDGE 3/5LT,ICE CREAM GELATO MACKINAC FDGE,1
8,ICE CREAM GELATO MACKINAC FDGE 3/5LT,ICE CREAM COOKIE%CRM DUTCH FUD,0
9,ICE CREAM GELATO MACKINAC FDGE 3/5LT,ICE CREAM GELATO BUTR PECAN,0


In [6]:
len(df)

79023

In [22]:
!pip install transformers sentence-transformers datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00


In [7]:
from sklearn.model_selection import train_test_split
from sentence_transformers import CrossEncoder
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.metrics import accuracy_score
import torch

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
# 5. Prepare CrossEncoder model
from sentence_transformers import CrossEncoder

# Load the model
model_name = "cross-encoder/stsb-roberta-base"
cross_encoder = CrossEncoder(model_name, num_labels=1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [24]:
import torch.optim as optim
from sentence_transformers import InputExample
from torch.optim import AdamW
from transformers import AutoTokenizer

In [10]:
# Split into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert the DataFrame to InputExample format for CrossEncoder
train_examples = [InputExample(texts=[row['Invoice'], row['Purchase Order']], label=row['Label']) for _, row in train_df.iterrows()]
val_examples = [InputExample(texts=[row['Invoice'], row['Purchase Order']], label=row['Label']) for _, row in val_df.iterrows()]

In [18]:
from transformers import Trainer, TrainingArguments
from torch.nn.utils.rnn import pad_sequence

In [31]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [32]:
# Set up a custom data collator
def custom_data_collator(features):
    # Tokenize the texts
    texts = [feature.texts for feature in features]
    labels = [feature.label for feature in features]

    # Tokenize the input texts using the model's tokenizer
    tokenized_texts = tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )

    # Create labels tensor with float type
    return {
        "input_ids": tokenized_texts['input_ids'],
        "attention_mask": tokenized_texts['attention_mask'],
        "labels": torch.tensor(labels, dtype=torch.float)  # Specify dtype here
    }


In [35]:

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory for model predictions and checkpoints
    evaluation_strategy="epoch",     # Evaluation strategy to adopt during training
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=16,   # Batch size for training
    per_device_eval_batch_size=16,    # Batch size for evaluation
    num_train_epochs=4,              # Total number of training epochs
    weight_decay=0.01,               # Strength of weight decay
)

# Create a Trainer instance
trainer = Trainer(
    model=cross_encoder.model,       # Use the underlying model
    args=training_args,              # Training arguments, defined above
    train_dataset=train_examples,    # Training dataset
    eval_dataset=val_examples,       # Evaluation dataset
    data_collator=custom_data_collator  # Use custom data collator
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [36]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.022300,0.020233
2,0.016800,0.015015


Epoch,Training Loss,Validation Loss
1,0.022300,0.020233
2,0.016800,0.015015
3,0.009900,0.014515
4,0.007900,0.012570


TrainOutput(global_step=15808, training_loss=0.015074571587054835, metrics={'train_runtime': 2839.1603, 'train_samples_per_second': 89.066, 'train_steps_per_second': 5.568, 'total_flos': 4871832586188060.0, 'train_loss': 0.015074571587054835, 'epoch': 4.0})

In [37]:
# Evaluate the model
eval_results = trainer.evaluate()

# Print evaluation results
print("Evaluation results:", eval_results)

# Extract accuracy from evaluation results (if available)
if 'eval_accuracy' in eval_results:
    print(f"Validation Accuracy: {eval_results['eval_accuracy']:.2f}")
else:
    print("Accuracy not found in evaluation results.")

Evaluation results: {'eval_loss': 0.012570305727422237, 'eval_runtime': 41.0137, 'eval_samples_per_second': 385.359, 'eval_steps_per_second': 24.09, 'epoch': 4.0}
Accuracy not found in evaluation results.


In [39]:
from sklearn.metrics import accuracy_score
import numpy as np

# Make predictions on the validation set
val_predictions = []
val_labels = []

# Create a DataLoader for the validation set
val_dataloader = DataLoader(val_examples, batch_size=8, shuffle=False, collate_fn=custom_collate_fn)

# Set the underlying model to evaluation mode
cross_encoder.model.eval()

# Iterate through the validation DataLoader
with torch.no_grad():
    for batch in val_dataloader:
        tokenized_texts, labels = batch

        # Move inputs and labels to the same device (GPU/CPU)
        tokenized_texts = {key: val.to(cross_encoder.model.device) for key, val in tokenized_texts.items()}
        labels = labels.to(cross_encoder.model.device)

        # Forward pass with tokenized inputs
        outputs = cross_encoder.model(**tokenized_texts)

        # Get predictions (using a threshold of 0.5 for binary classification)
        predictions = outputs.logits.cpu().numpy()
        val_predictions.extend(np.round(predictions).astype(int).flatten())  # Convert logits to binary predictions
        val_labels.extend(labels.cpu().numpy().flatten())  # Flatten and append labels

# Calculate accuracy
accuracy = accuracy_score(val_labels, val_predictions)

print(f"Validation Accuracy: {accuracy:.2f}")

Validation Accuracy: 0.99


In [40]:
model_save_path = '/content/drive/MyDrive/Invoice_PO_Match/in_po_model'

# Save the model
trainer.save_model(model_save_path)

# Optionally save the tokenizer as well (in case you need it later)
tokenizer.save_pretrained(model_save_path)

('/content/drive/MyDrive/Invoice_PO_Match/in_po_model/tokenizer_config.json',
 '/content/drive/MyDrive/Invoice_PO_Match/in_po_model/special_tokens_map.json',
 '/content/drive/MyDrive/Invoice_PO_Match/in_po_model/vocab.json',
 '/content/drive/MyDrive/Invoice_PO_Match/in_po_model/merges.txt',
 '/content/drive/MyDrive/Invoice_PO_Match/in_po_model/added_tokens.json',
 '/content/drive/MyDrive/Invoice_PO_Match/in_po_model/tokenizer.json')

In [45]:
import torch
from sentence_transformers import CrossEncoder

# Load the saved model
model_save_path = '/content/drive/MyDrive/Invoice_PO_Match/in_po_model'
cross_encoder = CrossEncoder(model_save_path)

def map_invoices_to_purchase_orders(invoices, purchase_orders):
    # Create text pairs for all combinations
    pairs = [(invoice, po) for invoice in invoices for po in purchase_orders]

    # Run inference
    predictions = cross_encoder.predict(pairs, convert_to_numpy=True)

    # Create mapping based on predictions
    result_mapping = []
    num_purchase_orders = len(purchase_orders)

    # Iterate through invoices and find the best matching purchase order
    for i in range(len(invoices)):
        # Get the scores for the current invoice against all purchase orders
        start_index = i * num_purchase_orders
        end_index = start_index + num_purchase_orders
        best_match_index = predictions[start_index:end_index].argmax()
        best_match_po = purchase_orders[best_match_index]
        result_mapping.append({invoices[i]: best_match_po})

    return result_mapping

# Example data with realistic product names
invoices = [
    "Laptop Model X, 16GB RAM, 512GB SSD",
    "Wireless Mouse, Ergonomic Design",
    "Office Chair, Adjustable Height, Black",
    "HD Monitor, 27 inch, 144Hz"
]

purchase_orders = [
    "Office Chair, Black, Ergonomic",
    "Wireless Mouse, Bluetooth, White",
    "Laptop Model X, 16GB RAM",
    "HD Monitor, 27 inch, 144Hz"
]

# Get the mapping
mapping_result = map_invoices_to_purchase_orders(invoices, purchase_orders)
print(mapping_result)

[{'Laptop Model X, 16GB RAM, 512GB SSD': 'Laptop Model X, 16GB RAM'}, {'Wireless Mouse, Ergonomic Design': 'Office Chair, Black, Ergonomic'}, {'Office Chair, Adjustable Height, Black': 'Office Chair, Black, Ergonomic'}, {'HD Monitor, 27 inch, 144Hz': 'HD Monitor, 27 inch, 144Hz'}]


In [46]:
import torch
from sentence_transformers import CrossEncoder

# Load the saved model
model_save_path = '/content/drive/MyDrive/Invoice_PO_Match/in_po_model'
cross_encoder = CrossEncoder(model_save_path)

def map_invoices_to_purchase_orders(invoices, purchase_orders):
    # Create text pairs for all combinations
    pairs = [(invoice, po) for invoice in invoices for po in purchase_orders]

    # Run inference
    predictions = cross_encoder.predict(pairs, convert_to_numpy=True)

    # Create mapping based on predictions
    result_mapping = []
    num_purchase_orders = len(purchase_orders)

    # Iterate through invoices and find the best matching purchase order
    for i in range(len(invoices)):
        # Get the scores for the current invoice against all purchase orders
        start_index = i * num_purchase_orders
        end_index = start_index + num_purchase_orders
        best_match_index = predictions[start_index:end_index].argmax()
        best_match_po = purchase_orders[best_match_index]
        result_mapping.append({invoices[i]: best_match_po})

    return result_mapping

# Example data with realistic product names
invoices = ['Jersey Mikes Mini Wheat Sub', 'Jersey Mikes', 'Jersey Mikes Wheat Sub', 'Jersey Mikes White Sub']

purchase_orders = ['ROLL MINI WHEAT SUB', 'ROLL MINI WHT SUB', 'ROLL WHEAT SUB', 'ROLL WHITE SUB']

# Get the mapping
mapping_result = map_invoices_to_purchase_orders(invoices, purchase_orders)
mapping_result

[{'Jersey Mikes Mini Wheat Sub': 'ROLL MINI WHEAT SUB'},
 {'Jersey Mikes': 'ROLL WHITE SUB'},
 {'Jersey Mikes Wheat Sub': 'ROLL WHEAT SUB'},
 {'Jersey Mikes White Sub': 'ROLL WHITE SUB'}]